In [7]:
from wallstreet import wallstreet

In [8]:
stock = wallstreet.Stock('AAPL')
df = stock.historical(days_back=5)
print(df)

        Date        Open        High         Low       Close   Adj Close  \
0 2022-08-22  169.690002  169.860001  167.139999  167.570007  167.570007   
1 2022-08-23  167.080002  168.710007  166.649994  167.229996  167.229996   
2 2022-08-24  167.320007  168.110001  166.250000  167.529999  167.529999   
3 2022-08-25  168.779999  170.139999  168.350006  170.029999  170.029999   
4 2022-08-26  170.570007  171.039993  164.000000  164.089996  164.089996   

     Volume  
0  69026800  
1  54147100  
2  53841500  
3  51218200  
4  54641909  


In [2]:
from sqlalchemy import MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from setup_psql_enironment import get_database

In [3]:
def create_database():
    # Setup environment and create a session
    db = get_database()
    Session = sessionmaker(bind=db)
    meta = MetaData(bind=db)
    session = Session()

In [4]:
db = get_database()
Session = sessionmaker(bind=db)
meta = MetaData(bind=db)
session = Session()

Connected to PostgreSQL database!


In [ ]:
with db.connect() as connection:
    with connection.begin():
        tickers_with_id = connection.execute("SELECT ticker, id FROM security").fetchall()
        for ticker, _id in tickers_with_id:
            stock = wallstreet.Stock(ticker)
            df = stock.historical(days_back=5)
            print(df)
            for index, row in df.iterrows():
                pass
                # print(row['Open'])
                # connection.execute("INSERT INTO daily_price (data_vendor_id, ticker_id, price_date, open_price, high_price, low_price, close_price, adj_close_price, volume) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")

print(tickers_with_id[:5])

In [6]:
with db.connect() as connection:
    with connection.begin():
        tickers_with_id = connection.execute("SELECT ticker, id FROM security").fetchall()
        
print(tickers_with_id)


2022-09-13 17:48:06,690 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-13 17:48:06,690 INFO sqlalchemy.engine.Engine SELECT ticker, id FROM security
2022-09-13 17:48:06,690 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-09-13 17:48:06,705 INFO sqlalchemy.engine.Engine COMMIT
[('A', 1), ('AA', 2), ('AAC', 3), ('AACG', 4), ('AACI', 5), ('AACIU', 6), ('AACIW', 7), ('AADI', 8), ('AAIC', 9), ('AAIC^B', 10), ('AAIN', 11), ('AAL', 12), ('AAM^A', 13), ('AAM^B', 14), ('AAMC', 15), ('AAME', 16), ('AAN', 17), ('AAOI', 18), ('AAON', 19), ('AAP', 20), ('AAPL', 21), ('AAQC', 22), ('AAT', 23), ('AATC', 24), ('AAU', 25), ('AAWW', 26), ('AB', 27), ('ABB', 28), ('ABBV', 29), ('ABC', 30), ('ABCB', 31), ('ABCL', 32), ('ABCM', 33), ('ABEO', 34), ('ABEV', 35), ('ABG', 36), ('ABGI', 37), ('ABIO', 38), ('ABM', 39), ('ABMD', 40), ('ABNB', 41), ('ABOS', 42), ('ABR', 43), ('ABR^E', 44), ('ABSI', 45), ('ABST', 46), ('ABT', 47), ('ABTX', 48), ('ABUS', 49), ('ABVC', 50), ('AC', 51), ('ACA', 52), ('ACAB', 5